In [135]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef
import scipy.stats as stats

In [136]:
#Functions

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    roc = roc_auc_score(y_test, predictions)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%'.format(accuracy))
    print(f'AUC = {roc}')
    return accuracy

In [137]:
features = pd.read_csv('../Data/training_set_features.csv')
labels = pd.read_csv('../Data/training_set_labels.csv')

df = pd.merge(features, labels, on='respondent_id', how='inner')

df['sex'] = df['sex'].map({
    'Female': 0,
    'Male': 1
})

df['marital_status'] = df['marital_status'].map({
    'Not Married': 0,
    'Married': 1
})

df['rent_or_own'] = df['rent_or_own'].map({
    'Rent': 0,
    'Own': 1
})
for col in df.columns:
    print(col, df[col].unique())

respondent_id [    0     1     2 ... 26704 26705 26706]
h1n1_concern [ 1.  3.  2.  0. nan]
h1n1_knowledge [ 0.  2.  1. nan]
behavioral_antiviral_meds [ 0.  1. nan]
behavioral_avoidance [ 0.  1. nan]
behavioral_face_mask [ 0.  1. nan]
behavioral_wash_hands [ 0.  1. nan]
behavioral_large_gatherings [ 0.  1. nan]
behavioral_outside_home [ 1.  0. nan]
behavioral_touch_face [ 1.  0. nan]
doctor_recc_h1n1 [ 0. nan  1.]
doctor_recc_seasonal [ 0. nan  1.]
chronic_med_condition [ 0.  1. nan]
child_under_6_months [ 0.  1. nan]
health_worker [ 0.  1. nan]
health_insurance [ 1. nan  0.]
opinion_h1n1_vacc_effective [ 3.  5.  4.  2.  1. nan]
opinion_h1n1_risk [ 1.  4.  3.  2.  5. nan]
opinion_h1n1_sick_from_vacc [ 2.  4.  1.  5.  3. nan]
opinion_seas_vacc_effective [ 2.  4.  5.  3.  1. nan]
opinion_seas_risk [ 1.  2.  4.  3.  5. nan]
opinion_seas_sick_from_vacc [ 2.  4.  1.  5. nan  3.]
age_group ['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']
education ['< 12 Years' '

In [138]:
categorical_columns = [
    #'education',
    #'income_poverty',
    #'marital_status',
    #'rent_or_own',
    #'employment_industry',
    #'employment_occupation',
    'sex',
    #'hhs_geo_region',
    'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    'child_under_6_months',
    'chronic_med_condition',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
]

numerical_columns = [
    'household_children',
    'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    #'health_insurance'
]

binary_columns = [
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'rent_or_own',
    'marital_status',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
    'child_under_6_months',
    'chronic_med_condition'
]

non_binary_columns = [
    'age_group',
    'race',
    'income_poverty',
    'employment_status',
    'hhs_geo_region',
    'census_msa',
]

targets = ['h1n1_vaccine', 'seasonal_vaccine']

df_nona = df.dropna()

In [139]:
#H1N1 Binary Correlations
dict = []
for x in binary_columns:
    binary = df_nona[x]
    target = df_nona['h1n1_vaccine']
    coef = {'Column': x,
           'Corr': matthews_corrcoef(binary,target)}
    dict.append(coef)
    
pd.DataFrame(dict).sort_values(by='Corr')

,Column,Corr
2,behavioral_antiviral_meds,0.020191
3,behavioral_outside_home,0.032933
4,behavioral_large_gatherings,0.032941
6,behavioral_avoidance,0.054768
11,child_under_6_months,0.073705
8,marital_status,0.081329
7,rent_or_own,0.083579
5,behavioral_touch_face,0.089193
0,behavioral_face_mask,0.090719
12,chronic_med_condition,0.106065


In [140]:
h1n1_cols = [
    'child_under_6_months',
    'marital_status',
    'rent_or_own',
    'behavioral_touch_face',
    'behavioral_face_mask',
    'chronic_med_condition',
    'behavioral_wash_hands',
    'doctor_recc_seasonal',
    'doctor_recc_h1n1'
]

In [141]:
#Seasonal Binary Correlations
dict = []
for x in binary_columns:
    binary = df_nona[x]
    target = df_nona['seasonal_vaccine']
    coef = {'Column': x,
           'Corr': matthews_corrcoef(binary,target)}
    dict.append(coef)
    
pd.DataFrame(dict).sort_values(by='Corr')

,Column,Corr
2,behavioral_antiviral_meds,-0.000019
11,child_under_6_months,0.025204
4,behavioral_large_gatherings,0.034773
3,behavioral_outside_home,0.035823
0,behavioral_face_mask,0.048361
6,behavioral_avoidance,0.069538
8,marital_status,0.094946
5,behavioral_touch_face,0.118539
12,chronic_med_condition,0.130180
7,rent_or_own,0.133144


In [142]:
seas_cols = [
    'marital_status',
    'behavioral_touch_face',
    'chronic_med_condition',
    'rent_or_own',
    'behavioral_wash_hands',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal'
]

In [143]:
#H1N1 Pearson Correlations
dict = []
for x in numerical_columns:
    numeric = df_nona[x]
    target = df_nona['h1n1_vaccine']
    coef = {'Column': x,
           'Corr': stats.pearsonr(numeric,target)[0],
           'P-Value': stats.pearsonr(numeric, target)[1]}
    dict.append(coef)
    
pd.DataFrame(dict).sort_values(by='Corr')

,Column,Corr,P-Value
9,opinion_seas_sick_from_vacc,-0.017852,1.521210e-01
1,household_adults,-0.004492,7.186137e-01
0,household_children,0.004623,7.107485e-01
6,opinion_h1n1_sick_from_vacc,0.108546,2.482173e-18
2,h1n1_concern,0.189103,6.840825e-53
3,h1n1_knowledge,0.203852,2.390794e-61
7,opinion_seas_vacc_effective,0.245726,3.785689e-89
5,opinion_h1n1_vacc_effective,0.318694,6.856173e-152
8,opinion_seas_risk,0.373196,7.091897e-212
4,opinion_h1n1_risk,0.460778,0.000000e+00


In [144]:
h1n1_num = ['opinion_h1n1_sick_from_vacc',
           'h1n1_concern',
           'h1n1_knowledge',
           'opinion_seas_vacc_effective',
           'opinion_h1n1_vacc_effective',
           'opinion_seas_risk',
           'opinion_h1n1_risk']

In [145]:
#Seasonal Pearson Correlations
dict = []
for x in numerical_columns:
    numeric = df_nona[x]
    target = df_nona['seasonal_vaccine']
    coef = {'Column': x,
           'Corr': stats.pearsonr(numeric,target)[0],
           'P-Value': stats.pearsonr(numeric, target)[1]}
    dict.append(coef)
    
pd.DataFrame(dict).sort_values(by='Corr')

,Column,Corr,P-Value
0,household_children,-0.048643,9.453367e-05
9,opinion_seas_sick_from_vacc,-0.033458,7.260898e-03
1,household_adults,-0.013618,2.746298e-01
6,opinion_h1n1_sick_from_vacc,0.046805,1.722969e-04
3,h1n1_knowledge,0.197707,9.653973e-58
2,h1n1_concern,0.199643,7.265152e-59
5,opinion_h1n1_vacc_effective,0.249553,5.618909e-92
4,opinion_h1n1_risk,0.316242,1.812387e-149
7,opinion_seas_vacc_effective,0.372235,1.033152e-210
8,opinion_seas_risk,0.489885,0.000000e+00


In [146]:
seas_num = [
    'h1n1_knowledge',
    'h1n1_concern',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_risk',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk'
]

In [147]:
#H1N1 Categorical Non-Binary
dict = []
for x in non_binary_columns:
    binary = df_nona[x].astype('category').cat.codes
    target = df_nona['h1n1_vaccine']
    coef = {'Column': x,
           'Corr': matthews_corrcoef(binary,target)}
    dict.append(coef)
    
pd.DataFrame(dict).sort_values(by='Corr')

C:\Users\mcand\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


,Column,Corr
5,census_msa,-0.004540
3,employment_status,0.000000
1,race,0.009087
0,age_group,0.013360
4,hhs_geo_region,0.015163
2,income_poverty,0.086730


In [155]:
h1n1_non_bin_cols = ['age_group', 'hhs_geo_region', 'income_poverty']

In [149]:
#Seasonal Categorical Non-Binary
dict = []
for x in non_binary_columns:
    binary = df_nona[x].astype('category').cat.codes
    target = df_nona['seasonal_vaccine']
    coef = {'Column': x,
           'Corr': matthews_corrcoef(binary,target)}
    dict.append(coef)
    
pd.DataFrame(dict).sort_values(by='Corr')

C:\Users\mcand\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


,Column,Corr
5,census_msa,-0.007506
3,employment_status,0.000000
1,race,0.004834
4,hhs_geo_region,0.006832
0,age_group,0.025665
2,income_poverty,0.101768


In [150]:
seas_non_bin_cols = ['age_group', 'income_poverty']

In [151]:
df_nona[x].astype('category').cat.codes

1        0
7        2
10       0
11       2
15       0
        ..
26681    0
26683    0
26690    1
26699    0
26703    1
Length: 6437, dtype: int8

In [152]:
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].mean())

health_worker_mean = df['health_worker'].mean()
behavioral_face_mask_mean = df['health_worker'].mean()
behavioral_wash_hands_mean = df['behavioral_wash_hands'].mean()
behavioral_antiviral_meds_mean = df['behavioral_antiviral_meds'].mean()
behavioral_outside_home_mean = df['behavioral_outside_home'].mean()
behavioral_large_gatherings_mean = df['behavioral_large_gatherings'].mean()
behavioral_touch_face_mean = df['behavioral_touch_face'].mean()
behavioral_avoidance_mean = df['behavioral_avoidance'].mean()
child_under_6_months_mean = df['child_under_6_months'].mean()
chronic_med_condition_mean = df['chronic_med_condition'].mean()
doctor_recc_h1n1_mean = df['doctor_recc_h1n1'].mean()
doctor_recc_seasonal_mean = df['doctor_recc_h1n1'].mean()

df['health_worker'] = df['health_worker'].fillna(health_worker_mean)
df['behavioral_face_mask'] = df['behavioral_face_mask'].fillna(behavioral_face_mask_mean)
df['behavioral_wash_hands'] = df['behavioral_wash_hands'].fillna(behavioral_wash_hands_mean)
df['behavioral_antiviral_meds'] = df['behavioral_antiviral_meds'].fillna(behavioral_antiviral_meds_mean)
df['behavioral_outside_home'] = df['behavioral_outside_home'].fillna(behavioral_outside_home_mean)
df['behavioral_large_gatherings'] = df['behavioral_large_gatherings'].fillna(behavioral_large_gatherings_mean)
df['behavioral_touch_face'] = df['behavioral_touch_face'].fillna(behavioral_touch_face_mean)
df['behavioral_avoidance'] = df['behavioral_avoidance'].fillna(behavioral_avoidance_mean)
df['child_under_6_months'] = df['child_under_6_months'].fillna(child_under_6_months_mean)
df['chronic_med_condition'] = df['chronic_med_condition'].fillna(chronic_med_condition_mean)
df['doctor_recc_h1n1'] = df['doctor_recc_h1n1'].fillna(doctor_recc_h1n1_mean)
df['doctor_recc_seasonal'] = df['doctor_recc_seasonal'].fillna(doctor_recc_seasonal_mean)

all_cols = categorical_columns + numerical_columns + targets
h1n1_cols = h1n1_cols + h1n1_num + h1n1_non_bin_cols
seas_cols = seas_cols + seas_num + seas_non_bin_cols
df = df[all_cols]

In [153]:
h1n1_cols

['child_under_6_months',
 'marital_status',
 'rent_or_own',
 'behavioral_touch_face',
 'behavioral_face_mask',
 'chronic_med_condition',
 'behavioral_wash_hands',
 'doctor_recc_seasonal',
 'doctor_recc_h1n1',
 'opinion_h1n1_sick_from_vacc',
 'h1n1_concern',
 'h1n1_knowledge',
 'opinion_seas_vacc_effective',
 'opinion_h1n1_vacc_effective',
 'opinion_seas_risk',
 'opinion_h1n1_risk',
 'age_group',
 'hss_geo_region',
 'income_poverty']

In [154]:
seas_cols

['marital_status',
 'behavioral_touch_face',
 'chronic_med_condition',
 'rent_or_own',
 'behavioral_wash_hands',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'h1n1_knowledge',
 'h1n1_concern',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'age_group',
 'income_poverty']

# Preprocessing

In [70]:
X_h1n1 = df[[]]
y_h1n1 = df[['h1n1_vaccine']]

C:\Users\mcand\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [222]:
#Categorical
cat_df = X[categorical_columns]
recat_df = pd.get_dummies(data=cat_df)

#Numerical
num_df = X[numerical_columns]

scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

encoded_df = pd.concat([recat_df, scaled_num_df], axis=1)

In [223]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_df, y, test_size=0.3, random_state=42)

y_train_h1n1 = y_train['h1n1_vaccine']
y_train_seas = y_train['seasonal_vaccine']

y_test_h1n1 = y_test['h1n1_vaccine']
y_test_seas = y_test['seasonal_vaccine']

In [224]:
#Create arrays
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

y_train_h1n1 = np.asarray(y_train_h1n1)
y_test_h1n1 = np.asarray(y_test_h1n1)

y_train_seas = np.asarray(y_train_seas)
y_test_seas = np.asarray(y_test_seas)

# Random Forest

In [225]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor()
model.fit(X_train, y_train_h1n1)

y_pred_h1n1 = model.predict(X_test)

In [226]:
#Set up RandomSearch
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 100, num=11)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf, 
                               param_distributions=random_grid, 
                               n_iter=50, 
                               cv=3, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs=-1)

### H1N1

In [227]:
#model = rf_random
model_h1n1 = RandomForestRegressor(n_estimators=800,
                              min_samples_split=2,
                              min_samples_leaf=4,
                              max_features='sqrt',
                              max_depth=20,
                              bootstrap=False)
model_h1n1.fit(X_train, y_train_h1n1)

RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_leaf=4, n_estimators=800)

In [228]:
#model.best_params_

In [229]:
base_model = RandomForestRegressor(n_estimators=10, random_state=42)
base_model.fit(X_train, y_train_h1n1)
evaluate(base_model, X_test, y_test_h1n1)

Model Performance
Average Error: 0.2483 degrees
Accuracy = nan%
AUC = 0.7816175070667295


<ipython-input-216-e8fb442df30f>:6: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / y_test)
<ipython-input-216-e8fb442df30f>:6: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * np.mean(errors / y_test)


nan

In [230]:
#best_random = rf_random.best_estimator_
evaluate(model_h1n1, X_test, y_test_h1n1)

Model Performance
Average Error: 0.2423 degrees
Accuracy = -inf%
AUC = 0.8301045010895534


<ipython-input-216-e8fb442df30f>:6: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / y_test)


-inf

In [231]:
y_predicted_h1n1 = model_h1n1.predict(X_test)

### Seasonal

In [232]:
#model = rf_random
model_seas = RandomForestRegressor(n_estimators=800,
                              min_samples_split=2,
                              min_samples_leaf=4,
                              max_features='sqrt',
                              max_depth=20,
                              bootstrap=False)

model_seas.fit(X_train, y_train_seas)

RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_leaf=4, n_estimators=800)

In [233]:
#model.best_params_

In [234]:
base_model = RandomForestRegressor(n_estimators=10, random_state=42)
base_model.fit(X_train, y_train_seas)
base_accuracy = evaluate(base_model, X_test, y_test_seas)

Model Performance
Average Error: 0.3251 degrees
Accuracy = nan%
AUC = 0.8082471249693057


<ipython-input-216-e8fb442df30f>:6: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / y_test)
<ipython-input-216-e8fb442df30f>:6: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * np.mean(errors / y_test)


In [235]:
evaluate(model_seas, X_test, y_test_seas)

Model Performance
Average Error: 0.3228 degrees
Accuracy = -inf%
AUC = 0.8529219770128039


<ipython-input-216-e8fb442df30f>:6: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / y_test)


-inf

In [236]:
y_predicted_seas = model_seas.predict(X_test)

### Combine Targets

In [237]:
y_predicted = np.vstack((y_predicted_h1n1, y_predicted_seas)).T
y_predicted

array([[0.19288598, 0.29934187],
       [0.26939945, 0.19817748],
       [0.06233589, 0.71491743],
       ...,
       [0.0611781 , 0.10010754],
       [0.08652902, 0.13497373],
       [0.10255067, 0.85995094]])

In [238]:
roc_auc_score(y_test, y_predicted)

0.8415132390511786

# Transform Test Data

In [160]:
df_full = pd.read_csv('../Data/test_set_features.csv')

df = df_full.drop(columns=['employment_occupation', 'employment_industry', 'health_insurance', 'respondent_id'])

In [161]:
categorical_columns = [
    #'education',
    #'income_poverty',
    #'marital_status',
    #'rent_or_own',
    #'employment_industry',
    #'employment_occupation',
    'sex',
    #'hhs_geo_region',
    #'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    #'child_under_6_months',
    'chronic_med_condition',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
]

numerical_columns = [
    #'household_children',
    #'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    #'health_insurance'
]


In [162]:
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].mean())

health_worker_mean = df['health_worker'].mean()
behavioral_face_mask_mean = df['health_worker'].mean()
behavioral_wash_hands_mean = df['behavioral_wash_hands'].mean()
behavioral_antiviral_meds_mean = df['behavioral_antiviral_meds'].mean()
behavioral_outside_home_mean = df['behavioral_outside_home'].mean()
behavioral_large_gatherings_mean = df['behavioral_large_gatherings'].mean()
behavioral_touch_face_mean = df['behavioral_touch_face'].mean()
behavioral_avoidance_mean = df['behavioral_avoidance'].mean()
child_under_6_months_mean = df['child_under_6_months'].mean()
chronic_med_condition_mean = df['chronic_med_condition'].mean()
doctor_recc_h1n1_mean = df['doctor_recc_h1n1'].mean()
doctor_recc_seasonal_mean = df['doctor_recc_h1n1'].mean()

df['health_worker'] = df['health_worker'].fillna(health_worker_mean)
df['behavioral_face_mask'] = df['behavioral_face_mask'].fillna(behavioral_face_mask_mean)
df['behavioral_wash_hands'] = df['behavioral_wash_hands'].fillna(behavioral_wash_hands_mean)
df['behavioral_antiviral_meds'] = df['behavioral_antiviral_meds'].fillna(behavioral_antiviral_meds_mean)
df['behavioral_outside_home'] = df['behavioral_outside_home'].fillna(behavioral_outside_home_mean)
df['behavioral_large_gatherings'] = df['behavioral_large_gatherings'].fillna(behavioral_large_gatherings_mean)
df['behavioral_touch_face'] = df['behavioral_touch_face'].fillna(behavioral_touch_face_mean)
df['behavioral_avoidance'] = df['behavioral_avoidance'].fillna(behavioral_avoidance_mean)
df['child_under_6_months'] = df['child_under_6_months'].fillna(child_under_6_months_mean)
df['chronic_med_condition'] = df['chronic_med_condition'].fillna(chronic_med_condition_mean)
df['doctor_recc_h1n1'] = df['doctor_recc_h1n1'].fillna(doctor_recc_h1n1_mean)
df['doctor_recc_seasonal'] = df['doctor_recc_seasonal'].fillna(doctor_recc_seasonal_mean)

full_cols = categorical_columns + numerical_columns
df = df[full_cols]

# Preprocessing

In [163]:
X = df.copy()

In [164]:
#Categorical
cat_df = X[categorical_columns]
recat_df = pd.get_dummies(data=cat_df)

#Numerical
num_df = X[numerical_columns]

scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

encoded_df = pd.concat([recat_df, scaled_num_df], axis=1)
X = np.asarray(encoded_df)

In [165]:
results_h1n1 = model_h1n1.predict(X)
results_seas = model_seas.predict(X)

results = np.vstack((results_h1n1, results_seas)).T

In [166]:
results = pd.DataFrame(results, columns=['h1n1_vaccine', 'seasonal_vaccine'])

submission = pd.concat([df_full, results], axis=1)
submission = submission[['respondent_id', 'h1n1_vaccine', 'seasonal_vaccine']]

In [167]:
submission.to_csv('../Submissions/Submission 7.1.21.csv', index=False)